In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import hdf5storage

# GPS file path
#gps_path = "/kucresis/scratch/tteisberg_sta/scripts/python/utig_radar_loading/outputs/gps/2018_Antarctica_BaslerJKB/gps_20190114_1.mat"
#gps_path = "/kucresis/scratch/tteisberg_sta/scripts/python/utig_radar_loading/outputs/gps/2018_Antarctica_BaslerJKB/gps_20190117_1.mat"
#gps_path = "/kucresis/scratch/tteisberg_sta/scripts/python/utig_radar_loading/outputs/gps/2018_Antarctica_BaslerJKB/gps_20190119_1.mat"
#gps_path = "/kucresis/scratch/tteisberg_sta/scripts/python/utig_radar_loading/outputs/gps/2017_Antarctica_BaslerJKB/gps_20171229_3.mat"
#gps_path = "/kucresis/scratch/tteisberg_sta/scripts/python/utig_radar_loading/outputs/gps/2017_Antarctica_BaslerJKB/gps_20180110_3.mat"
#gps_path = "/kucresis/scratch/tteisberg_sta/scripts/python/utig_radar_loading/outputs/gps/2015_Antarctica_BaslerJKB/gps_20160123_3.mat"
gps_path = "/kucresis/scratch/tteisberg_sta/scripts/python/utig_radar_loading/outputs/gps/2008_Antarctica_BaslerJKB/gps_20090317_1.mat"
#gps_path = "outputs/tmp/gps_test.mat"

print(f"Loading GPS file: {gps_path}")
print(f"File exists: {Path(gps_path).exists()}")

In [ ]:
gps_data = hdf5storage.loadmat(gps_path)

print("Keys in GPS data:")
for key in gps_data.keys():
    print(key)

print(f"GPS source: {gps_data['gps_source']}")

In [ ]:
for k in gps_data.keys():
    print(f"== {k} ==")
    print(f"  Type: {gps_data[k].dtype}")
    print(f"  Shape: {gps_data[k].shape}")
    # Check if is numeric before min/max
    if np.issubdtype(gps_data[k].dtype, np.number):
        print(f"  Min: {np.nanmin(gps_data[k])}")
        print(f"  Max: {np.nanmax(gps_data[k])}")
        print(f"  # NaNs: {np.sum(np.isnan(gps_data[k]))} out of {gps_data[k].size}")

In [ ]:
# Convert comp_time to datetime as seconds since unix epoch
unix_epoch = np.datetime64('1970-01-01T00:00:00')
comp_time = unix_epoch + np.array((gps_data['comp_time']), dtype='timedelta64[s]').squeeze()
print(f"comp_time range: {comp_time[0]} to {comp_time[-1]} ({comp_time[-1] - comp_time[0]})")

gps_time = unix_epoch + np.array((gps_data['gps_time']), dtype='timedelta64[s]').squeeze()
print(f"gps_time range: {gps_time[0]} to {gps_time[-1]} ({gps_time[-1] - gps_time[0]})")

radar_time = np.array((gps_data['radar_time']), dtype=np.float64).squeeze()
print(f"radar_time range: {radar_time[0]} to {radar_time[-1]} ({radar_time[-1] - radar_time[0]})")


In [ ]:
np.diff(gps_data['radar_time'][0,-10:])

In [ ]:
fig, ax = plt.subplots()
gps_time_deltas = np.diff(gps_data['gps_time'][0,:])
ax.plot(gps_time_deltas, label='gps_time_deltas')
comp_time_deltas = np.diff(gps_data['comp_time'][0,:])
#ax.plot(comp_time_deltas, label='comp_time_deltas')
radar_time_deltas = np.diff(gps_data['radar_time'][0,:])
ax.plot(radar_time_deltas, label='radar_time_deltas')
#ax.set_ylim(np.minimum(0, 1.1*np.min(gps_time_deltas)), np.max(gps_time_deltas) * 1.1)
ax.legend()
ax.set_xlim(1, len(gps_time_deltas)-10)
#ax.set_ylim(0, 0.2)

In [ ]:
gps_time_deltas[:10]

In [ ]:
fig, ax = plt.subplots()

ax.plot(gps_data['gps_time'][0,:], gps_data['lat'][0,:], label='Latitude')

In [ ]:
x_axis_variable = 'gps_time'
y_axis_variables = ['lat', 'lon', 'elev', 'heading', 'roll', 'pitch', 'radar_time']

fig, axs = plt.subplots(len(y_axis_variables), 1, figsize=(10, 2 * len(y_axis_variables)), sharex=True)

for ax, var in zip(axs, y_axis_variables):
    ax.plot(gps_data[x_axis_variable][0,:], gps_data[var][0,:])
    ax.set_ylabel(var)
    ax.grid()

axs[-1].set_xlabel(x_axis_variable)
plt.tight_layout()